In [151]:
import pandas as pd
import sqlite3

In [152]:
file_path = 'D:\\Nam_4\\Test_momo\\MoMo Talent 2024_DA_Case Study Round_Questions.xlsx'
transactions_df = pd.read_excel(file_path, sheet_name='Data Transactions')
commission_df = pd.read_excel(file_path, sheet_name='Data Commission')
user_info_df = pd.read_excel(file_path, sheet_name='Data User_Info')

# Xử lý cột Amount để chuyển đổi dấu phân cách hàng nghìn
transactions_df['Amount'] = transactions_df['Amount'].str.replace(',', '').astype(int)

# Kết nối tới SQLite database (có thể là một file hoặc trong bộ nhớ)
conn = sqlite3.connect('momo_data.db')

In [153]:
# Chuyển DataFrame thành bảng SQL
transactions_df.to_sql('Transactions', conn, if_exists='replace', index=False)
commission_df.to_sql('Commission', conn, if_exists='replace', index=False)
user_info_df.to_sql('User_Info', conn, if_exists='replace', index=False)


13428

In [154]:
# 1. Thêm cột 'Revenue' vào bảng 'Transactions' và tính tổng doanh thu của MoMo trong tháng 1 năm 2020
query = '''
    SELECT T.*, (T.Amount * (C.Rate_pct / 100.00)) as revenue
    FROM Transactions T
    JOIN Commission C ON T.Merchant_id = C.Merchant_id
'''
transactions_with_revenue = pd.read_sql_query(query, conn)
transactions_with_revenue.to_sql('Transactions_with_Revenue', conn, if_exists='replace', index=False)

13495

In [155]:
# Tính tổng doanh thu trong tháng 1 năm 2020
query = '''
    SELECT SUM(revenue) as total_revenue
    FROM Transactions_with_Revenue
    WHERE strftime('%Y-%m', date) = '2020-01'
'''
total_revenue_jan_2020 = pd.read_sql_query(query, conn)
print('Total Revenue in January 2020:', total_revenue_jan_2020['total_revenue'].iloc[0])


Total Revenue in January 2020: 1409827.02


In [156]:
# #cau1:
# query1 = '''
# ALTER TABLE Transactions ADD COLUMN revenue DECIMAL;
# UPDATE Transactions
# SET revenue = (
#     SELECT Amount * Rate_pct *100
#     FROM Commission,Transactions
#     WHERE Transactions.Merchant_id = Commission.Merchant_id
# );


# '''

# conn.execute("PRAGMA foreign_keys = OFF;")  # Tắt kiểm tra khóa ngoại để ALTER TABLE
# conn.executescript(query1)
# result1 = pd.read_sql_query("SELECT SUM(revenue) AS total_revenue FROM Transactions WHERE strftime('%Y-%m', date) = '2020-01';", conn)
# print(result1)


In [157]:
# 2. Tháng có lợi nhuận cao nhất của MoMo
query = '''
    SELECT strftime('%Y-%m', Date) as month, SUM(revenue) as total_revenue
    FROM Transactions_with_Revenue
    GROUP BY month
    ORDER BY total_revenue DESC
    LIMIT 1
'''
most_profitable_month = pd.read_sql_query(query, conn)
print('Most Profitable Month:', most_profitable_month['month'].iloc[0])

Most Profitable Month: 2020-10


In [158]:
# 3. Ngày trong tuần MoMo kiếm được nhiều tiền nhất và ít tiền nhất (trung bình)
query = '''
    SELECT strftime('%w', Date) as weekday, AVG(revenue) as avg_revenue
    FROM Transactions_with_Revenue
    GROUP BY weekday
    ORDER BY avg_revenue DESC
'''
revenue_by_weekday = pd.read_sql_query(query, conn)
print('Day with Most Revenue (on average):', revenue_by_weekday.iloc[0]['weekday'])
print('Day with Least Revenue (on average):', revenue_by_weekday.iloc[-1]['weekday'])


Day with Most Revenue (on average): 3
Day with Least Revenue (on average): 0


In [159]:
# 4. Kết hợp với bảng 'User_Info', thêm các cột 'Age', 'Gender', 'Location', 'Type_user' vào bảng 'Transactions' và tính tổng số người dùng mới trong tháng 12 năm 2020
query = '''
    SELECT T.*, U.age, U.gender, U.location,
           CASE
               WHEN strftime('%Y-%m', T.Date) = strftime('%Y-%m', U.first_tran_date) THEN 'New'
               ELSE 'Current'
           END as type_user
    FROM Transactions_with_Revenue T
    JOIN User_Info U ON T.user_id = U.user_id
'''
transactions_with_user_info = pd.read_sql_query(query, conn)
transactions_with_user_info.to_sql('Transactions_with_User_Info', conn, if_exists='replace', index=False)

# Tính tổng số người dùng mới trong tháng 12 năm 2020
query = '''
    SELECT COUNT(DISTINCT user_id) as new_users
    FROM Transactions_with_User_Info
    WHERE strftime('%Y-%m', Date) = '2020-12' AND type_user = 'New'
'''
new_users_dec_2020 = pd.read_sql_query(query, conn)
print('Total New Users in December 2020:', new_users_dec_2020['new_users'].iloc[0])

Total New Users in December 2020: 72


In [160]:
import sqlite3
import pandas as pd

# Kết nối đến cơ sở dữ liệu SQLite
conn = sqlite3.connect('momo_data.db')

# Lấy danh sách các bảng trong cơ sở dữ liệu
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = res.fetchall()

# In thông tin của từng bảng
for table in tables:
    table = table[0]
    df = pd.read_sql_query(f"SELECT * from {table}", conn)
    print(f"Table: {table}")
    print(df.head())  # In 5 dòng đầu tiên của mỗi bảng
    print("\n")

# Đóng kết nối
conn.close()

Table: Transactions
    user_id    order_id        Date  Amount  Merchant_id Purchase_status
0  21269588  4169517626  2020-01-01   10000           13            None
1  28097592  4170276686  2020-01-01   20000           13            None
2  47435144  4166729310  2020-01-01   10000           12            None
3  29080935  4174460303  2020-01-01   10000           13            None
4  14591075  4168216749  2020-01-01   10000           12            None


Table: Commission
  Merchant_name  Merchant_id  Rate_pct
0       Viettel           12         2
1      Mobifone           13         3
2     Vinaphone           14         4
3  Vietnamobile           15         4
4       Gmobile           16         4


Table: User_Info
    User_id First_tran_date      Location       Age  Gender
0  41654498      2018-05-02          HCMC  33_to_37    MALE
1  51276281      2019-12-08  Other Cities   unknown    MALE
2  49152375      2019-12-20  Other Cities  23_to_27    MALE
3   5971050      2019-08-01  